In [1]:
import pandas as pd

# import data
colors=pd.read_csv('colors.csv')
set_part=pd.read_csv('set_part.csv')
parts=pd.read_csv('parts.csv')
part_categories=pd.read_csv('part_categories.csv')
sets=pd.read_csv('sets.csv')
themes=pd.read_csv('themes.csv')

In [2]:
# merge all files into one using foreign key
df=set_part.merge(sets, left_on='set_id', right_on='set_id', how='left')
df=df.merge(parts, left_on='part_id', right_on='part_id', how='left')
df=df.merge(themes, left_on='theme_id', right_on='theme_id', how='left')
df=df.merge(colors, left_on='color_id', right_on='color_id', how='left')
df=df.merge(part_categories, left_on='part_cat_id', right_on='part_cat_id', how='left')
set_theme=sets.merge(themes,left_on='theme_id', right_on='theme_id', how='left')

In [3]:
dict_theme={}
dict_set={}

for i in range(len(set_theme)):
    dict_set[set_theme['set_id'][i]]=set_theme['set_name'][i]
    dict_theme[set_theme['set_id'][i]]=set_theme['theme_name'][i]
    


set_part=set_part.set_index('set_id')
set_part['num_part']=set_part.groupby('set_id')['part_id'].value_counts().unstack().T.sum()
set_part=set_part.reset_index()
set_part['theme']=set_part['set_id'].map(lambda x:dict_theme[x])

In [4]:
# exclude those in basic theme
set_part2=set_part[(set_part['theme']!='Basic Set')&(set_part['theme']!='Supplemental')&(set_part['theme']!='Basic Model')&(set_part['theme']!='Classic Basic')]
# keep only those sets with more than 5 kinds of parts
set_part2=set_part2[set_part2['num_part']>5]
# sp_q:set_part pivot table with quantity for each pair as the value
sp_q=set_part2.groupby(['set_id','part_id'])['quantity'].sum().unstack()
sp_q=sp_q.fillna(0)

In [5]:
sp_q.head()

part_id,0687b1,1,10016414,10019stk01,10026stk01,10029stk01,10036stk01,10039,10048,10049,...,x979,x979pb01,x988,xleash,xleash3,zbb013,zbb014,zbb015,zbb018,zbb022
set_id,,,,,,,,,,,,,,,,,,,,,
00-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00-2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00-3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00-4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0016-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
dict_rec={}
def fun1(x):
    '''
    modify the difference
    '''
    if x<0:
        x=abs(x)
    else:
        x=0
    return int(x)
    
    
def diff(set_id1, set_id2):
    '''
    calculate difference between two sets
    '''
    d=sp_q.loc[set_id1]-sp_q.loc[set_id2]
    d2=d.map(lambda x:fun1(x))
    return d2

def min_diff(set_id):
    '''
    return recommened set_ids
    '''
    set_all=pd.Series(sp_q.index)
    set_all=set_all[set_all!=set_id]
    diffs=set_all.map(lambda x:diff(set_id,x).sum())
    if sum(diffs==diffs.min())>3:
        return list(set_all[diffs==diffs.min()])
    else:
        return list(set_all[diffs.sort_values()[:3].index])

In [7]:
def recommend():
    '''
    output: the recommendations for the input set_id
    '''
    set_id=input("What's your set_id needed recommendation?\n")
    print()
    print('Here are the recommenations for Set %s.\n' % dict_set[set_id])
    if set_id not in dict_rec:
        dict_rec[set_id]=min_diff(set_id)
    for i, x in enumerate(dict_rec[set_id]):
        print('Recommendation %s:' % (i+1))
        print('Set %s of Theme %s' % (dict_set[x], dict_theme[x]))
        s=diff(set_id,x)
        if s.sum()==0:
            print('Extra 0 parts needed.')
        else:
            print('Extra %s part(s) needed:' % int(s.sum()))
            print(s[s>0])
        print()

In [8]:
set_id=list(sp_q.index)
set_id

['00-1',
 '00-2',
 '00-3',
 '00-4',
 '0016-1',
 '03093-1',
 '1-10',
 '1-8',
 '100-1',
 '100-2',
 '10000-1',
 '10001-1',
 '10002-1',
 '10009-1',
 '10010-1',
 '10011-1',
 '10012-1',
 '10013-1',
 '10014-1',
 '10015-1',
 '10016-1',
 '10017-1',
 '10018-1',
 '10019-1',
 '10020-1',
 '10020-2',
 '10021-1',
 '10022-1',
 '10023-1',
 '10024-1',
 '10025-1',
 '10026-1',
 '10027-1',
 '10029-1',
 '10030-1',
 '10036-1',
 '10037-1',
 '10039-1',
 '10040-1',
 '10041-1',
 '10042-1',
 '10044-1',
 '10048-1',
 '10049-1',
 '10056-1',
 '10057-1',
 '10058-1',
 '10059-1',
 '10060-1',
 '10061-1',
 '10062-1',
 '10063-1',
 '10064-1',
 '10065-1',
 '10066-1',
 '10067-1',
 '10068-1',
 '10069-1',
 '10070-1',
 '10071-1',
 '10072-1',
 '10073-1',
 '10074-1',
 '10075-1',
 '10076-1',
 '10077-1',
 '10079-1',
 '10080-1',
 '10081-1',
 '10082-1',
 '10083-1',
 '10090-1',
 '10106-1',
 '10116-1',
 '10117-1',
 '10118-1',
 '10121-1',
 '10123-1',
 '10124-1',
 '10127-1',
 '10128-1',
 '10129-1',
 '10131-1',
 '10132-1',
 '10133-1',
 '10

In [9]:
#u can try any set_id in sp_q indexs, eg: 00-1, 00-2, 00-3, 00-4, 0016-1, etc. (but the set_id is not in arrangement)
recommend()

What's your set_id needed recommendation?
00-1

Here are the recommenations for Set Weetabix Castle.

Recommendation 1:
Set Life of George II of Theme Other
Extra 17 part(s) needed:
part_id
10218    14
3005      2
3622      1
dtype: int64

Recommendation 2:
Set Advent Calendar 2002 Creator (Day 20) Steamship of Theme Creator
Extra 17 part(s) needed:
part_id
3021      1
3062b     1
3660      1
4070     14
dtype: int64

Recommendation 3:
Set Orange - Suntory Promotional of Theme Food & Drink
Extra 18 part(s) needed:
part_id
3001    2
3024    3
3660    7
6141    6
dtype: int64

